<a href="https://colab.research.google.com/github/Dhaneshkp/NLP/blob/main/fork-of-llm-classification-finetuning-dhanesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
submission_test = pd.read_csv("/content/sample_submission.csv")
submission_test

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/config.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/tokenizer.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/metadata.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/model.weights.h5
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/assets/tokenizer/vocabulary.spm


In [2]:


#!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

#model = SentenceTransformer("all-mpnet-base-v2")


In [3]:

#model.save("/kaggle/working/all-mpnet-base-v2")


In [4]:
from transformers import AutoModel, AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/qwen-llm")
#model = AutoModel.from_pretrained("/kaggle/input/qwen-llm")



In [5]:
df_train = pd.read_csv("/content/train.csv")
df_train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [6]:
df_test = pd.read_csv("/content/test.csv")
df_test

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [8]:
df_train=df_train.head(10000)

In [9]:
df_train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
9995,743558482,gpt-3.5-turbo-0613,gpt-4-1106-preview,"[""Describe chebyshev polynomials""]","[""Chebyshev polynomials, named after the Russi...","[""Chebyshev polynomials are a sequence of orth...",1,0,0
9996,743759177,chatglm3-6b,gemini-pro-dev-api,"[""Ddr5 at 6000mt\/s has a bandwidth of x GB\/s...","[""To determine the bandwidth of a DDR5 memory ...","[""1. Bandwidth is calculated by multiplying th...",0,0,1
9997,743813489,openhermes-2.5-mistral-7b,gpt-4-1106-preview,"[""I have to run a relay on a team with Usain B...","[""It would be best for Usain Bolt to take the ...","[""In a relay team with Usain Bolt, who is know...",0,0,1
9998,743891198,gpt-4-0613,gpt-4-1106-preview,"[""How about?""]","[""Of course! How can I assist you today?""]","[""It seems like your message may have been cut...",0,0,1


In [10]:
df_train.isnull().sum()

,0
id,0
model_a,0
model_b,0
prompt,0
response_a,0
response_b,0
winner_model_a,0
winner_model_b,0
winner_tie,0


In [11]:
X = df_train.drop(['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie'], axis = 1)
X

,id,prompt,response_a,response_b
0,30192,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o..."
1,53567,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat..."
2,65089,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ..."
3,96401,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c..."
4,198779,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru..."
...,...,...,...,...
9995,743558482,"[""Describe chebyshev polynomials""]","[""Chebyshev polynomials, named after the Russi...","[""Chebyshev polynomials are a sequence of orth..."
9996,743759177,"[""Ddr5 at 6000mt\/s has a bandwidth of x GB\/s...","[""To determine the bandwidth of a DDR5 memory ...","[""1. Bandwidth is calculated by multiplying th..."
9997,743813489,"[""I have to run a relay on a team with Usain B...","[""It would be best for Usain Bolt to take the ...","[""In a relay team with Usain Bolt, who is know..."
9998,743891198,"[""How about?""]","[""Of course! How can I assist you today?""]","[""It seems like your message may have been cut..."


In [12]:
y = df_train[['winner_model_a', 'winner_model_b', 'winner_tie']].values

y = np.argmax(y, axis=1)
y

array([0, 1, 2, ..., 2, 2, 1])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

In [14]:
X_train

,id,prompt,response_a,response_b
9254,689927743,"[""My daughter who is 19 years old, and have -1...","[""I'm not an eye doctor, but I can provide som...","[""Prisms in glasses are prescribed to correct ..."
1561,117188139,"[""How\u2019s the weather today?""]","[""As an AI language model, I don't have real-t...","[""As an AI, I'm unable to provide real-time in..."
1670,126227049,"[""What is an AI Agent?""]","[""An AI agent is a software program or a syste...","[""An AI agent is a system that interacts with ..."
6087,457054687,"[""Describe the story of Cinderella in exactly ...","[""Abandoned by her parents, Cinderella endures...","[""Abandoned beauty, Cinderella danced elegantl..."
6669,501589422,"[""In Julia using XLSX.jl package I want to loa...","[""To load a sheet from an XLSX file in Julia u...","[""To load a sheet in Julia using the XLSX.jl p..."
...,...,...,...,...
5734,432877900,"[""How to earn money with exchange offices that...","[""Here are some tips for earning money with ex...","[""While I don't have access to specific Reddit..."
5191,392856958,"[""Please translate into german:\n\nThis joint ...","[""Dieser gemeinsame FEDS-Brief und Liberty Str...","[""Dieser gemeinsame FEDS-Hinweis und Liberty S..."
5390,407589571,"[""Write a physical therapy 8-12 sentence parag...","[""The patient, a 76-year-old male, has been re...","[""Physical Therapy Assessment:\n\nPatient is a..."
860,62750940,"[""How to wear a bikini?""]","[""Wearing a bikini can be a fun and comfortabl...","[""Here are the basic steps to wear a bikini:\n..."


In [15]:
X_test

,id,prompt,response_a,response_b
6252,468242541,"[""Cook instant noodles "",""What if I Boil water...","[""Cooking instant noodles is a quick and easy ...","[""Here are the basic steps to cook instant noo..."
4684,353989191,"[""What are the most important commands and des...","[""Here are some important commands and descrip...","[""Here are some of the most important Git and ..."
1731,131590828,"[""x^2 - 5x+6 = 0 \nSolve for x, use Factoring ...","[""x * x - 5 * x + 6 = 0\n(x - 5) * x + 6 * (-1...","[""Sure, I can help you solve this equation usi..."
4742,358536354,"[""how much money can you make selling lemonade...","[""Depending on how much money you invest upfro...","[""It is difficult to give an exact answer to t..."
4521,341763586,"[""How can a man go eight days without sleep?\n...","[""By sleeping at night.""]","[""By sleeping at night and therefore not needi..."
...,...,...,...,...
6412,480885090,"[""where is a good place to rock climb in India...","[""There are many rock climbing locations in In...","[""One of the best places to rock climb in Indi..."
8285,621278659,"[""Who is reckful""]","[""Reckful, also known as Byron Bernstein, was ...","[""Reckful, whose real name was Byron Bernstein..."
7853,588180192,"[""I am a 40 years old man. What are the risks ...","[""Here are some potential risks of playing ten...","[""Playing tennis on a weekly basis can be a gr..."
1095,81238423,"[""d + a + b + c = a, b * a = 1, b =3 , d * c =...","[""Given the equations:\n\n1) d + a + b + c = a...","[""From the given equations, we can deduce the ..."


In [16]:
y_train

array([2, 2, 2, ..., 1, 1, 2])

In [17]:
catagorical_feature = [col for col in X.columns if X[col].dtype == 'object']
catagorical_feature

['prompt', 'response_a', 'response_b']

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch

from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="all-mpnet-base-v2"):
        self.model_name = model_name  
        self.model = SentenceTransformer(model_name)
        if torch.cuda.is_available():
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X.tolist(), show_progress_bar=False, device="cuda" if torch.cuda.is_available() else "cpu")


# Example usage in ColumnTransformer
preprocessing = ColumnTransformer([
    ("prompt_embed", HFEmbedder(), "prompt"),
    ("resp_a_embed", HFEmbedder(), "response_a"),
    ("resp_b_embed", HFEmbedder(), "response_b"),
    ("num", "passthrough", ["id"])
])


from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/open-minilm-l6-v2",use_auth_token=False , local_files_only=True,batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.model = SentenceTransformer(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            embeddings = self.model.encode(
                batch,
                show_progress_bar=False,
                device="cuda" if self.use_cuda else "cpu"
            )
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)



In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_deberta_v3_model(config):
    # Extract config values
    vocab_size = config["vocabulary_size"]
    num_layers = config["num_layers"]
    num_heads = config["num_heads"]
    hidden_dim = config["hidden_dim"]
    intermediate_dim = config["intermediate_dim"]
    dropout_rate = config["dropout"]
    max_seq_len = config["max_sequence_length"]

    # Input layers
    input_ids = layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_ids")
    attention_mask = layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="attention_mask")

    # Embedding layer
    embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=hidden_dim)(input_ids)

    # Positional encoding (simplified)
    position_embeddings = layers.Embedding(input_dim=max_seq_len, output_dim=hidden_dim)(tf.range(start=0, limit=max_seq_len, delta=1))
    position_embeddings = tf.expand_dims(position_embeddings, axis=0)
    x = embedding_layer + position_embeddings

    # Transformer blocks
    for _ in range(num_layers):
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_dim)(x, x, attention_mask=attention_mask)
        attention_output = layers.Dropout(dropout_rate)(attention_output)
        attention_output = layers.LayerNormalization()(x + attention_output)

        ffn_output = layers.Dense(intermediate_dim, activation='gelu')(attention_output)
        ffn_output = layers.Dense(hidden_dim)(ffn_output)
        ffn_output = layers.Dropout(dropout_rate)(ffn_output)
        x = layers.LayerNormalization()(attention_output + ffn_output)

    # Output (for embedding use, you might just return x)
    pooled_output = layers.GlobalAveragePooling1D()(x)

    model = models.Model(inputs=[input_ids, attention_mask], outputs=pooled_output)
    return model


In [20]:
import kagglehub

# Download latest version
path = kagglehub.model_download("keras/deberta_v3/keras/deberta_v3_extra_small_en")

print("Path to model files:", path)

Path to model files: /kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3


In [21]:
from transformers import AutoTokenizer, AutoModel
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModel
from transformers import DebertaV2Tokenizer
from tensorflow.keras.models import load_model
import torch
import numpy as np
from keras_hub.src.models.deberta_v3.deberta_v3_backbone import DebertaV3Backbone

config = {
    "vocabulary_size": 128100,
    "num_layers": 12,
    "num_heads": 6,
    "hidden_dim": 384,
    "intermediate_dim": 1536,
    "dropout": 0.1,
    "max_sequence_length": 512,
    "bucket_size": 256
}


class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3", batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.tokenizer = DebertaV2Tokenizer(vocab_file="/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/assets/tokenizer/vocabulary.spm")
        #self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        #self.model = AutoModel.from_pretrained(self.model_path)
        self.model =  DebertaV3Backbone(
            vocabulary_size=128100,
            num_layers=12,
            num_heads=6,
            hidden_dim=384,
            intermediate_dim=1536,
            dropout=0.1,
            max_sequence_length=512,
            bucket_size=256
        )
  # Define your model architecture here
        self.model.load_weights("/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3/model.weights.h5")
        print([input.name for input in self.model.inputs])

        if self.use_cuda:
            self.model = self.model#.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt")

            inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt")

            # Rename keys to match model input expectations
            model_inputs = {
                "padding_mask": inputs["attention_mask"],
                "token_ids": inputs["input_ids"]
            }

            if self.use_cuda:
                model_inputs = {k: v.to("cuda") for k, v in model_inputs.items()}

            with torch.no_grad():
                outputs = self.model(model_inputs)

            # Mean pooling over token embeddings
            embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)


In [22]:
from transformers import DebertaV2Tokenizer
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
import tensorflow as tf
from keras_hub.src.models.deberta_v3.deberta_v3_backbone import DebertaV3Backbone
import os # Import os module

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path, batch_size=4, max_length=512, use_fast_tokenizer=True, enable_mixed_precision=True):
        import tensorflow as tf
        import numpy as np
        import os
        import multiprocessing

        self.model_path = model_path
        self.batch_size = int(batch_size)
        self.max_length = min(int(max_length), 512)
        self.use_fast_tokenizer = use_fast_tokenizer
        self.enable_mixed_precision = enable_mixed_precision # Add this line to store the parameter
        self._cpu_count = multiprocessing.cpu_count()

        # Make TF GPU usage explicit / safe
        try:
            gpus = tf.config.list_physical_devices("GPU")
            if gpus:
                for g in gpus:
                    tf.config.experimental.set_memory_growth(g, True)
        except Exception:
            pass

        # optionally use mixed precision on GPUs (speeds up fp16 capable GPUs)
        try:
            if self.enable_mixed_precision: # Use self.enable_mixed_precision
                from tensorflow.keras import mixed_precision
                mixed_precision.set_global_policy("mixed_float16")
        except Exception:
            pass

        # Load tokenizer (prefer fast tokenizer if available)
        try:
            if self.use_fast_tokenizer:
                from transformers import DebertaV2TokenizerFast as _TokFast
                self.tokenizer = _TokFast(vocab_file=os.path.join(self.model_path, "assets/tokenizer/vocabulary.spm")) # Use os.path.join
            else:
                from transformers import DebertaV2Tokenizer as _Tok
                self.tokenizer = _Tok(vocab_file=os.path.join(self.model_path, "assets/tokenizer/vocabulary.spm")) # Use os.path.join
        except Exception:
            # fallback to original import name/location
            try:
                from transformers import DebertaV2Tokenizer as _Tok
                self.tokenizer = _Tok(vocab_file=os.path.join(self.model_path, "assets/tokenizer/vocabulary.spm")) # Use os.path.join
            except Exception:
                raise

        self.tokenizer.model_max_length = self.max_length

        # Load model backbone (Keras)
        self.model = DebertaV3Backbone(
            vocabulary_size=128100,
            num_layers=12,
            num_heads=6,
            hidden_dim=384,
            intermediate_dim=1536,
            dropout=0.1,
            max_sequence_length=512,
            bucket_size=256
        )
        self.model.load_weights(os.path.join(self.model_path, "model.weights.h5")) # Use os.path.join

        print("✅ Model input names:", [input.name for input in self.model.inputs])
        # optional: warm-up call with zeros to ensure TF places variables on GPU if available
        try:
            import tensorflow as tf
            dummy_input = {
                "padding_mask": tf.zeros((1, self.max_length), dtype=tf.int32),
                "token_ids": tf.zeros((1, self.max_length), dtype=tf.int32),
            }
            _ = self.model(dummy_input)
        except Exception:
            pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        import tensorflow as tf
        import numpy as np

        # Accept pandas Series / DataFrame / numpy array / list
        if hasattr(X, "to_list"):
            texts = X.to_list()
        else:
            texts = list(X)

        # Coerce each row to a single text string.
        def _to_text(item):
            if isinstance(item, (list, tuple, np.ndarray)):
                if len(item) == 0:
                    return ""
                if len(item) == 1:
                    return str(item[0])
                return " ".join(str(x) for x in item)
            return "" if item is None else str(item)

        texts = [_to_text(t) for t in texts]
        n = len(texts)
        if n == 0:
            # no data -> return empty with inferred dim if possible
            try:
                hidden_dim = int(self.model.output_shape[-1])
                return np.zeros((0, hidden_dim), dtype=np.float32)
            except Exception:
                return np.empty((0,))

        # If tokenizer is "fast" we can tokenize the whole dataset in one call (fast, Rust-backed)
        try:
            if self.use_fast_tokenizer:
                # Vectorized tokenization to numpy arrays (fast)
                tokens_np = self.tokenizer(
                    texts,
                    padding="max_length",
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="np",
                )
                # Build a tf.data.Dataset to feed the model efficiently (reduces Python overhead)
                input_ids = tf.cast(tokens_np["input_ids"], tf.int32)
                attention_mask = tf.cast(tokens_np["attention_mask"], tf.int32)

                ds = tf.data.Dataset.from_tensor_slices(
                    {"token_ids": input_ids, "padding_mask": attention_mask}
                )
                ds = ds.batch(max(1, self.batch_size)).prefetch(tf.data.AUTOTUNE)

                # Use model.predict on the dataset so Keras can optimize execution and improve GPU utilization
                outputs = self.model.predict(ds, verbose=0)
                embeddings = np.mean(outputs, axis=1)
                return embeddings
        except Exception:
            # fall back to batching loop below if anything fails
            pass

        # Fallback: batch-tokenize and call model per-batch (keeps fewer copies)
        all_embeddings = []
        hidden_dim = None
        try:
            hidden_dim = int(self.model.output_shape[-1])
        except Exception:
            hidden_dim = None

        for i in range(0, n, self.batch_size):
            batch = texts[i:i + self.batch_size]
            # Tokenize directly to TensorFlow tensors where possible (reduces copies)
            try:
                tokens = self.tokenizer(
                    batch,
                    padding="max_length",
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="tf",
                )
                token_ids_tf = tf.cast(tokens["input_ids"], tf.int32)
                attention_mask_tf = tf.cast(tokens["attention_mask"], dtype=tf.int32)
                model_inputs = {"padding_mask": attention_mask_tf, "token_ids": token_ids_tf}
                outputs = self.model(model_inputs)
                embeddings = tf.reduce_mean(outputs, axis=1).numpy()
            except Exception:
                # safest fallback: numpy tokenization then tf convert
                tokens = self.tokenizer(
                    [str(t) for t in batch],
                    padding="max_length",
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="np",
                )
                token_ids_tf = tf.convert_to_tensor(tokens["input_ids"], dtype=tf.int32)
                attention_mask_tf = tf.convert_to_tensor(tokens["attention_mask"], dtype=tf.int32)
                model_inputs = {"padding_mask": attention_mask_tf, "token_ids": token_ids_tf}
                outputs = self.model(model_inputs)
                embeddings = tf.reduce_mean(outputs, axis=1).numpy()

            all_embeddings.append(embeddings)
            if hidden_dim is None:
                try:
                    hidden_dim = int(embeddings.shape[1])
                except Exception:
                    hidden_dim = None

        if not all_embeddings:
            if hidden_dim is not None:
                return np.zeros((0, hidden_dim), dtype=np.float32)
            return np.empty((0,))

        return np.vstack(all_embeddings)

In [23]:
import os
print(os.listdir("/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3"))


['config.json', 'assets', 'tokenizer.json', 'metadata.json', 'model.weights.h5']


In [24]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text_for_common_words(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


class CommonWordsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        common_meaningful_words_a = []
        common_meaningful_words_b = []

        for index, row in X.iterrows():
            prompt_tokens = clean_text_for_common_words(row['prompt'])
            response_a_tokens = clean_text_for_common_words(row['response_a'])
            response_b_tokens = clean_text_for_common_words(row['response_b'])

            common_meaningful_a = len(set(prompt_tokens) & set(response_a_tokens))
            common_meaningful_b = len(set(prompt_tokens) & set(response_b_tokens))

            common_meaningful_words_a.append(common_meaningful_a)
            common_meaningful_words_b.append(common_meaningful_b)

        return np.array([common_meaningful_words_a, common_meaningful_words_b]).T

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
embedder = HFEmbedder(model_path=path, enable_mixed_precision=True) # Explicitly pass enable_mixed_precision
X_train["prompt_clean"] = X_train["prompt"].apply(clean_text_for_common_words)
X_train["response_a_clean"] = X_train["response_a"].apply(clean_text_for_common_words)
X_train["response_b_clean"] = X_train["response_b"].apply(clean_text_for_common_words)

common_words_transformer = CommonWordsTransformer()
# Updated preprocessing transformer
preprocessing = ColumnTransformer([
    ("prompt_embed", embedder, "prompt_clean"),
    ("resp_a_embed", embedder, "response_a_clean"),
    ("resp_b_embed", embedder, "response_a_clean"),
    ("common_words", common_words_transformer, ["prompt", "response_a", "response_b"]),
    #("num", "passthrough", ["id"])
])

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Model input names: ['padding_mask', 'token_ids']


In [26]:
preprocessing

ColumnTransformer(transformers=[('prompt_embed',
                                 HFEmbedder(model_path='/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3'),
                                 'prompt_clean'),
                                ('resp_a_embed',
                                 HFEmbedder(model_path='/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3'),
                                 'response_a_clean'),
                                ('resp_b_embed',
                                 HFEmbedder(model_path='/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3'),
                                 'response_a_clean'),
                                ('common_words', CommonWordsTransformer(),
                                 ['prompt', 'response_a', 'response_b'])])

In [27]:
#feature_selection = SelectKBest(score_func = chi2, k=6)
from sklearn.feature_selection import SelectKBest, f_classif

feature_selection = SelectKBest(score_func=f_classif, k=6)


In [28]:
model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [29]:
my_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    #("feature_selection", feature_selection),
    ("model", model)
])
my_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('prompt_embed',
                                                  HFEmbedder(model_path='/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3'),
                                                  'prompt_clean'),
                                                 ('resp_a_embed',
                                                  HFEmbedder(model_path='/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3'),
                                                  'response_a_clean'),
                                                 ('resp_b_embed',
                                                  HFEmbedder(model_path='/kaggle/in...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None, num_class=3, ...))])

In [ ]:
scores = cross_val_score(my_pipeline, X_train, y_train, cv=3, scoring="accuracy")
print("Accuracy:", scores.mean())

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Model input names: ['padding_mask', 'token_ids']
✅ Model input names: ['padding_mask', 'token_ids']
✅ Model input names: ['padding_mask', 'token_ids']
✅ Model input names: ['padding_mask', 'token_ids']
✅ Model input names: ['padding_mask', 'token_ids']
✅ Model input names: ['padding_mask', 'token_ids']


In [ ]:
my_pipeline.fit(X_train, y_train)

In [ ]:
df_test["prompt_clean"] = df_test["prompt"].apply(clean_text_for_common_words)
df_test["response_a_clean"] = df_test["response_a"].apply(clean_text_for_common_words)
df_test["response_b_clean"] = df_test["response_b"].apply(clean_text_for_common_words)

In [ ]:
probs = my_pipeline.predict_proba(df_test)

submission = pd.DataFrame({
    "id": df_test["id"],
    "winner_model_a": probs[:,0],
    "winner_model_b": probs[:,1],
    "winner_tie": probs[:,2],
})
print(submission)
submission.to_csv("submission.csv", index=False)

In [ ]:
from transformers import DebertaV2Tokenizer
import torch
import numpy as np
from keras_hub.src.models.deberta_v3.deberta_v3_backbone import DebertaV3Backbone
import tensorflow as tf

# Configuration
model_path = "/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/3"
vocab_file = f"{model_path}/assets/tokenizer/vocabulary.spm"
weights_file = f"{model_path}/model.weights.h5"

# Initialize tokenizer
tokenizer = DebertaV2Tokenizer(vocab_file=vocab_file)
print("✅ Tokenizer initialized:", type(tokenizer))

# Initialize model
model = DebertaV3Backbone(
    vocabulary_size=128100,
    num_layers=12,
    num_heads=6,
    hidden_dim=384,
    intermediate_dim=1536,
    dropout=0.1,
    max_sequence_length=512,
    bucket_size=256
)
model.load_weights(weights_file)
print("✅ Model loaded and weights applied.")
print("Model input names:", [input.name for input in model.inputs])

# Example input text
text = "DeBERTa is a powerful transformer model."

# Tokenize the input
tokens = tokenizer(text, return_tensors="pt")
print("✅ Tokenizer output keys:", tokens.keys())
print("Input IDs:", tokens["input_ids"])
print("Attention Mask:", tokens["attention_mask"])

# Convert token IDs to token strings
input_ids = tokens["input_ids"][0]
token_names = tokenizer.convert_ids_to_tokens(input_ids)
print("✅ Token names:", token_names)

# Convert PyTorch tensors to NumPy arrays (on CPU)
padding_mask_np = tokens["attention_mask"].cpu().numpy()
token_ids_np = tokens["input_ids"].cpu().numpy()

# Convert to TensorFlow tensors
model_inputs = {
    "padding_mask": tf.convert_to_tensor(padding_mask_np),
    "token_ids": tf.convert_to_tensor(token_ids_np)
}

# Run forward pass
print("🚀 Running model forward pass...")
outputs = model(model_inputs)

# Inspect model output
print("✅ Model output type:", type(outputs))
if isinstance(outputs, dict):
    print("Model output keys:", outputs.keys())
    if "last_hidden_state" in outputs:
        hidden_state = outputs["last_hidden_state"]
        print("Last hidden state shape:", hidden_state.shape)
        # Mean pooling and convert to NumPy
        embeddings = tf.reduce_mean(hidden_state, axis=1).numpy()
        print("✅ Mean pooled embeddings shape:", embeddings.shape)
    else:
        print("⚠️ Warning: 'last_hidden_state' not found in model output.")
else:
    print("⚠️ Model returned a non-dict output:", outputs)
